In [1]:
import pandas as pd
import yfinance as yf
from sqlalchemy import create_engine
from ta import add_all_ta_features


In [21]:
aapl_data = yf.download("AAPL", period="max")


[*********************100%%**********************]  1 of 1 completed


In [24]:
# Convert Date to datetime
aapl_data["Date"] = pd.to_datetime(aapl_data.index)

# Calculate basic indicators
aapl_data["MA9"] = aapl_data["Close"].rolling(window=9).mean()
aapl_data["MA21"] = aapl_data["Close"].rolling(window=21).mean()
aapl_data["MA35"] = aapl_data["Close"].rolling(window=35).mean()

# Select relevant columns
aapl_data = aapl_data[["Date", "Open", "High", "Low", "Close", "Volume", "MA9", "MA21", "MA35"]]


C:\Users\vishv\AppData\Local\Temp\ipykernel_25444\3720966964.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aapl_data["Date"] = pd.to_datetime(aapl_data.index)
C:\Users\vishv\AppData\Local\Temp\ipykernel_25444\3720966964.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aapl_data["MA9"] = aapl_data["Close"].rolling(window=9).mean()
C:\Users\vishv\AppData\Local\Temp\ipykernel_25444\3720966964.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

In [25]:
# Replace with your database connection details
#only run once to create DB
#engine = create_engine("sqlite:///my_stock_data.db")


# Create table if not exists
aapl_data.to_sql("apple_stock", engine, if_exists="append", index=False)


10834

In [29]:
#sorted descending by date

aapl_data_ordered = pd.read_sql_query("SELECT * FROM apple_stock ORDER BY Date DESC", engine)


# Example 1: Find all days with RSI above 70 (overbought)
overbought_days = pd.read_sql_query(
    "SELECT * FROM apple_stock WHERE MA21 > 70", engine
)

# Example 2: Calculate average volume for days with MA20 above Close
avg_vol_above_ma20 = pd.read_sql_query(
    "SELECT * FROM apple_stock WHERE MA21 > Close", engine
)


In [31]:
# Create table if not exists
aapl_data_ordered.to_sql("apple_stock_ordered", engine, if_exists="append", index=False)

10834

In [28]:
#to kill or dispose the database
#engine.dispose()